In [3]:
import psycopg2

In [4]:
import mysql.connector

In [5]:
import pandas as pd

In [6]:
import psycopg2.extras as extras

In [7]:
import warnings

In [8]:
# source database
def get_conn_mysql():
    conn = mysql.connector.connect(host="localhost", port=3306, user="root", password="", db="sakila")
    # start a connection
    cur = conn.cursor()
    return cur, conn

In [9]:
# target database
def get_conn_postgresql():
    conn = psycopg2.connect(host="bronto.ewi.utwente.nl",database="dab_ds22231a_82",user="dab_ds22231a_82",password="xJk7LAOiN9Sjyffa")
    # start a connection
    cur = conn.cursor()
    return cur, conn

In [10]:
# loading the df into the target database tables
def execute_values(conn, df, table):

    tuples = [tuple(x) for x in df.to_numpy()]

    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    try:
        extras.execute_values(cur2, query, tuples)
        conn2.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        return 1
    print("-------data updated/inserted into table----",table)


In [11]:
warnings.simplefilter("ignore", UserWarning)

In [12]:
cur1, conn1 = get_conn_mysql()

In [13]:
cur2, conn2 = get_conn_postgresql()

In [14]:
cur2.execute("SET SEARCH_PATH TO project;")

In [14]:
query_customer = "select\
    cu.customer_id as customer_key,\
    cu.customer_id as customer_id,\
    cu.first_name as customer_first_name,\
    cu.last_name as customer_last_name,\
    cu.email as customer_email,\
    a.address as customer_address,\
    a.district as customer_district,\
    sakila.city.city as customer_city,\
    sakila.country.country as customer_country,\
    a.postal_code as customer_postal_code,\
    a.phone as customer_phone,\
    if(cu.active,\
    _utf8'active',\
    _utf8'') as customer_active from \
    (((sakila.customer cu \
join sakila.address a on \
    (cu.address_id = a.address_id)) \
join sakila.city on \
    (a.city_id = sakila.city.city_id)) \
join sakila.country on \
    (sakila.city.country_id = sakila.country.country_id))"

In [15]:
df1= pd.read_sql(query_customer,con=conn1)

In [16]:
execute_values(conn2, df1,'project.dim_customer')

-------data updated/inserted into table---- project.dim_customer


In [17]:
query_film = "select \
    sakila.film.film_id as film_key,\
    sakila.film.film_id as film_id, \
    sakila.film.title as film_title, \
    sakila.film.description as film_description, \
    sakila.film.release_year as film_release_year, \
    sakila.`language`.name  as film_language, \
    sakila.film.original_language_id as film_original_language, \
    sakila.film.rental_duration as film_rental_duration, \
    sakila.film.rental_rate as film_rental_rate, \
    sakila.film. length   as film_length, \
    sakila.film.replacement_cost as film_replacement_cost, \
    sakila.film.rating as film_rating, \
    sakila.film.special_features as film_special_feature, \
    sakila.category.name as film_category, \
    group_concat(concat(sakila.actor.first_name, _utf8' ',sakila.actor.last_name) separator ', ' ) as film_actors \
from \
    (((((sakila.category \
left join sakila.film_category on \
    (sakila.category.category_id = sakila.film_category.category_id)) \
left join sakila.film on \
    (sakila.film_category.film_id = sakila.film.film_id )) \
left join  sakila.film_actor  on \
    ( sakila.film.film_id  =  sakila.film_actor.film_id )) \
left join  sakila.actor  on \
    ( sakila.film_actor.actor_id  =  sakila.actor.actor_id )) \
join sakila.`language` on \
    (sakila.film.language_id = sakila.`language`.language_id)) \
group by \
     sakila.film.film_id, \
     sakila.category.name"

In [18]:
df2= pd.read_sql(query_film,con=conn1)

In [19]:
df2.film_special_feature = df2.film_special_feature.astype(str)

In [20]:
df2['film_special_feature'] = df2['film_special_feature'].str.replace('{','')

C:\Users\PLN\AppData\Local\Temp\ipykernel_3316\61376148.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['film_special_feature'] = df2['film_special_feature'].str.replace('{','')


In [21]:
df2['film_special_feature'] = df2['film_special_feature'].str.replace('}','')

C:\Users\PLN\AppData\Local\Temp\ipykernel_3316\181542963.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['film_special_feature'] = df2['film_special_feature'].str.replace('}','')


In [22]:
df2['film_special_feature'] = df2['film_special_feature'].str.replace("'",'')

In [23]:
df2['film_special_feature'] = df2['film_special_feature'].str.replace(", ",",")

In [24]:
df2.film_special_feature

0             Deleted Scenes,Behind the Scenes
1                      Trailers,Deleted Scenes
2                      Trailers,Deleted Scenes
3               Commentaries,Behind the Scenes
4                               Deleted Scenes
                        ...                   
995                 Trailers,Behind the Scenes
996                 Trailers,Behind the Scenes
997                             Deleted Scenes
998                    Trailers,Deleted Scenes
999    Trailers,Commentaries,Behind the Scenes
Name: film_special_feature, Length: 1000, dtype: object

In [25]:
execute_values(conn2, df2,'project.dim_film')

-------data updated/inserted into table---- project.dim_film


In [26]:
query_staff = "select \
    s.staff_id as staff_key, \
    s.staff_id as staff_id, \
    s.first_name as staff_first_name, \
    s.last_name as staff_last_name, \
    concat(s.first_name, _utf8' ', s.last_name) as staff_full_name, \
    s.store_id as staff_store_id, \
    (select a.address  from address a where a.address_id =  \
        (select s2.address_id from store s2 where s2.address_id = s.store_id)) as staff_store_name, \
    if(s.active, \
    _utf8'active', \
    _utf8'') as staff_active \
from sakila.staff s"


In [27]:
df3= pd.read_sql(query_staff,con=conn1)

In [28]:
execute_values(conn2, df3,'project.dim_staff')

-------data updated/inserted into table---- project.dim_staff


In [29]:
query_store = "select \
    s.store_id  as store_key, \
    s.store_id  as store_id, \
    a.address as store_address, \
    a.district  as store_district, \
    a.postal_code as store_postal_code, \
    a.phone as store_phone, \
    sakila.city.city as store_city, \
    sakila.country.country as store_country, \
    concat(st.first_name, _utf8' ', st.last_name) as store_manager \
    from \
    ((((sakila.store s \
    join sakila.address a on \
    (s.address_id = a.address_id)) \
    join sakila.city on \
    (a.city_id = sakila.city.city_id)) \
    join sakila.country on \
    (sakila.city.country_id = sakila.country.country_id)) \
    join sakila.staff st on \
    (st.staff_id = s.manager_staff_id))"

In [30]:
df4= pd.read_sql(query_store,con=conn1)

In [31]:
execute_values(conn2, df4,'project.dim_store')

-------data updated/inserted into table---- project.dim_store


In [32]:
query_rental = "select r.rental_id as rental_id, \
 r.customer_id as dim_customer_key, \
 i.film_id as film_key, \
 i.store_id as store_key, \
 r.staff_id as staff_key, \
 CAST(r.rental_date AS char) as rental_date, \
 CAST(r.return_date AS char) as return_date \
 from (sakila.rental r \
 join sakila.inventory i on \
 (i.inventory_id = r.inventory_id))"

In [33]:
df5= pd.read_sql(query_rental,con=conn1)

In [34]:
df5.rental_date = pd.to_datetime(df5.rental_date)

In [35]:
execute_values(conn2, df5,'project.fact_rental')

-------data updated/inserted into table---- project.fact_rental


In [15]:
query_payment = "select p.payment_id as payment_id, \
	   CAST(i.film_id as char) as film_key, \
	   p.staff_id as staff_key, \
	   CAST(i.store_id as char) as store_key, \
	   p.customer_id as customer_key, \
	   p.amount as payment_amount, \
       CAST(p.payment_date AS char) as payment_date, \
       CAST(r.rental_date as char) as rental_date, \
	   CAST(r.return_date as char) as return_date \
from ((sakila.payment p \
left join sakila.rental r on \
(r.rental_id = p.rental_id)) \
left join sakila.inventory i on \
(i.inventory_id = r.inventory_id))"

In [16]:
df6= pd.read_sql(query_payment,con=conn1)

In [81]:
df6.payment_date = pd.to_datetime(df6.payment_date)

In [17]:
df6

,payment_id,film_key,staff_key,store_key,customer_key,payment_amount,payment_date,rental_date,return_date
0,1,663,1,2,1,2.99,2005-05-25 11:30:37,2005-05-25 11:30:37,2005-06-03 12:00:37
1,2,875,1,2,1,0.99,2005-05-28 10:35:23,2005-05-28 10:35:23,2005-06-03 06:32:23
2,3,611,1,1,1,5.99,2005-06-15 00:54:12,2005-06-15 00:54:12,2005-06-23 02:42:12
3,4,228,2,2,1,0.99,2005-06-15 18:02:53,2005-06-15 18:02:53,2005-06-19 15:54:53
4,5,308,2,1,1,9.99,2005-06-15 21:08:46,2005-06-15 21:08:46,2005-06-25 02:26:46
...,...,...,...,...,...,...,...,...,...
16044,16045,1000,1,1,599,4.99,2005-08-21 17:43:42,2005-08-21 17:43:42,2005-08-22 18:53:42
16045,16046,891,1,2,599,1.99,2005-08-21 21:41:57,2005-08-21 21:41:57,2005-08-25 20:37:57
16046,16047,881,2,2,599,8.99,2005-08-23 06:09:44,2005-08-23 06:09:44,2005-09-01 06:53:44
16047,16048,869,2,2,599,2.99,2005-08-23 11:08:46,2005-08-23 11:08:46,2005-08-25 07:25:46


In [18]:
execute_values(conn2, df6,'project.fact_payment')

-------data updated/inserted into table---- project.fact_payment


In [100]:
query_inventory = "select i.inventory_id as inventory_id, \
                    cast(i.store_id as char) as store_key, \
                    cast(i.film_id as char) as film_key \
                    from sakila.inventory i "

In [101]:
df7= pd.read_sql(query_inventory,con=conn1)

In [102]:
df7

,inventory_id,store_key,film_key
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,16,1,4
...,...,...,...
4576,4573,2,999
4577,4578,2,1000
4578,4579,2,1000
4579,4580,2,1000


In [103]:
execute_values(conn2, df7,'project.fact_inventory')

-------data updated/inserted into table---- project.fact_inventory
